PyGSLIB
========

Introduction
---------------

This is a set of test for the drillhole functions.

These functions are not part of GSLIB but are required for some basic drillhole test and processing (example desurvey)



In [1]:
#general imports  
import pygslib as gslib      
import pandas as pd

#make the plots inline
%matplotlib inline  

Getting the data ready for work
---------

Here we use arbitrary data


In [2]:
collar = pd.DataFrame ({'bhid'   : [0,1],
                        'xcollar': [0.,10.],
                        'ycollar': [2., 8.], 
                        'zcollar': [5.,50.] 
                       })
survey = pd.DataFrame ({'bhid'   : [0,0,0,1,1,1],
                        'azi'    : [0.,10,20,100.,110,130],
                        'dip'    : [90,80,50,90,70,50],         # positive downdip
                        'len'    : [0,10,20,0,10,20] 
                       })
assay  = pd.DataFrame ({'bhid'   : [0,0,0,1,1,1],
                        'from'   : [ 10,20,30,50,60,80],
                        'to'     : [ 20,30,40,60,80,90]  
                       })

In [3]:
collar

,bhid,xcollar,ycollar,zcollar
0,0,0,2,5
1,1,10,8,50


In [4]:
survey

,azi,bhid,dip,len
0,0,0,90,0
1,10,0,80,10
2,20,0,50,20
3,100,1,90,0
4,110,1,70,10
5,130,1,50,20


In [5]:
assay

,bhid,from,to
0,0,10,20
1,0,20,30
2,0,30,40
3,1,50,60
4,1,60,80
5,1,80,90


## Testing validation functions issort

In [6]:
# test sort, overlap, gap and zero interval 
print gslib.__drillhole.issort.__doc__

issort = issort(cid,sid,sleng,aid,afrom,ato,tol,gaptol,inttol,[nc,ns,na])

Wrapper for ``issort``.

Parameters
----------
cid : input rank-1 array('i') with bounds (nc)
sid : input rank-1 array('i') with bounds (ns)
sleng : input rank-1 array('f') with bounds (ns)
aid : input rank-1 array('i') with bounds (na)
afrom : input rank-1 array('f') with bounds (na)
ato : input rank-1 array('f') with bounds (na)
tol : input float
gaptol : input float
inttol : input float

Other Parameters
----------------
nc : input int, optional
    Default: len(cid)
ns : input int, optional
    Default: len(sid)
na : input int, optional
    Default: len(aid)

Returns
-------
issort : int



In [7]:
parameter = {
    'cid'    : collar['bhid'],
    'sid'    : survey['bhid'],
    'sleng'  : survey['len'],
    'aid'    : assay['bhid'],
    'afrom'  : assay['from'],
    'ato'    : assay['to'],
    'tol'    : 0.01,
    'gaptol' : 2,
    'inttol' : 0.5}


print  'error : ', gslib.__drillhole.issort(**parameter), 'expected :', 0

error :  0 expected : 0


In [8]:
#test collar id not sorted by removing nin to max order
parameter['cid']= [1,0]
print  'error : ', gslib.__drillhole.issort(**parameter), 'expected :', 10
# fixing data again
parameter['cid']=collar.bhid

error :  10 expected : 10


In [9]:
#test survey id not sorted by removing nin to max order
parameter['sid']= [1,1,1,0,0,0]
print  'error : ', gslib.__drillhole.issort(**parameter), 'expected :', 20
# fixing data again
parameter['sid']=survey.bhid

error :  20 expected : 20


In [10]:
#test survey not sorted by len
parameter['sleng']= [0,20,10,0,10,20]
print  'error : ', gslib.__drillhole.issort(**parameter), 'expected :', 21
# fixing data again
parameter['sleng']=survey.len

error :  21 expected : 21


In [11]:
#test assay not sorted by id
parameter['aid']= [1,1,1,0,0,0]
print  'error : ', gslib.__drillhole.issort(**parameter), 'expected :', 30
# fixing data again
parameter['aid']=assay.bhid

error :  30 expected : 30


In [12]:
#test assay not sorted by from
parameter['afrom']= [ 30,20,30,50,60,80]
print  'error : ', gslib.__drillhole.issort(**parameter), 'expected :', 31
# fixing data again
parameter['afrom']=assay['from']

error :  31 expected : 31


In [13]:
#test assay gaps (within tolerance)
parameter['afrom']= [ 10,15,30,50,60,80]
parameter['ato']=   [ 20,30,40,60,80,90]
print  'error : ', gslib.__drillhole.issort(**parameter), 'expected :', 32
# fixing data again
parameter['afrom']=assay['from']
parameter['ato']=assay['to']

error :  32 expected : 32


In [14]:
#test assay overlap (within tolerance)
parameter['afrom']= [ 10,25,30,50,60,80]
parameter['ato']=   [ 20,30,40,60,80,90]
print  'error : ', gslib.__drillhole.issort(**parameter), 'expected :', 33
# fixing data again
parameter['afrom']=assay['from']
parameter['ato']=assay['to']

error :  33 expected : 33


In [15]:
#test assay from > to? (within tolerance)
parameter['afrom']= [ 10.1,10.1,30,50,60,80]
parameter['ato']=   [ 10.0,30,40,60,80,90]
print  'error : ', gslib.__drillhole.issort(**parameter), 'expected :', 34
# fixing data again
parameter['afrom']=assay['from']
parameter['ato']=assay['to']

error :  34 expected : 34


In [16]:
#test assay from - to > inttol? (interval too small )
parameter['afrom']= [ 10.0,10.1,30,50,60,80]
parameter['ato']=   [ 10.1,30,40,60,80,90]
print  'error : ', gslib.__drillhole.issort(**parameter), 'expected :', 35
# fixing data again
parameter['afrom']=assay['from']
parameter['ato']=assay['to']

error :  35 expected : 35


## Testing dmincurb (desurvey with minimum curvature)

In [17]:
print gslib.__drillhole.dmincurb.__doc__

ax,ay,az = dmincurb(cid,cx,cy,cz,sid,saz,sdip,sleng,aid,afrom,ato,[nc,ns,na])

Wrapper for ``dmincurb``.

Parameters
----------
cid : input rank-1 array('i') with bounds (nc)
cx : input rank-1 array('f') with bounds (nc)
cy : input rank-1 array('f') with bounds (nc)
cz : input rank-1 array('f') with bounds (nc)
sid : input rank-1 array('i') with bounds (ns)
saz : input rank-1 array('f') with bounds (ns)
sdip : input rank-1 array('f') with bounds (ns)
sleng : input rank-1 array('f') with bounds (ns)
aid : input rank-1 array('i') with bounds (na)
afrom : input rank-1 array('f') with bounds (na)
ato : input rank-1 array('f') with bounds (na)

Other Parameters
----------------
nc : input int, optional
    Default: len(cid)
ns : input int, optional
    Default: len(sid)
na : input int, optional
    Default: len(aid)

Returns
-------
ax : rank-1 array('f') with bounds (na)
ay : rank-1 array('f') with bounds (na)
az : rank-1 array('f') with bounds (na)



In [18]:
parameter_desurvey = {
    'cid'    : collar['bhid'],
    'cx'     : collar['xcollar'],
    'cy'     : collar['ycollar'],
    'cz'     : collar['zcollar'],
    'sid'    : survey['bhid'],
    'saz'    : survey['azi'],
    'sdip'   : survey['dip'],
    'sleng'  : survey['len'],
    'aid'    : assay['bhid'],
    'afrom'  : assay['from'],
    'ato'    : assay['to']}

ax,ay,az = gslib.__drillhole.dmincurb(**parameter_desurvey)

In [19]:
ax,ay,az

(array([ 0.,  0.,  0.,  0.,  0.,  0.], dtype=float32),
 array([ 0.,  0.,  0.,  0.,  0.,  0.], dtype=float32),
 array([ 0.,  0.,  0.,  0.,  0.,  0.], dtype=float32))